# 第５章　誤差逆伝搬法

第4章では数値微分と勾配法を使って、ニューラルネットワークの学習が可能であることを示しました。実際の学習では計算時間および計算精度の観点から数値微分は使われずに、ニューラルネットワークの解析的微分を効率よく計算するためのアルゴリズムである誤差逆伝搬法が使われます。この章では誤差逆伝搬法を学習します。

## 単純な関数の解析的微分

単純な関数の微分は解析的な表現が知られていることが多いです。

$$
\frac{d}{dx}x^n = nx^{n-1} \\
\frac{d}{dx}\sin(ax) = a\cos(ax) \\
\frac{d}{dx}\exp(ax) = a\exp(ax) \\
$$


## 合成関数の微分

では、複雑な関数の微分はどのように計算するのでしょうか。ここで二つの関数
$$
y = f(x) \\
z = g(y)
$$
が与えられていると、その合成関数$g\circ f$を考えます。
$$
(g\circ f)(x) = g(f(x))
$$

多くの複雑な関数は単純な関数の合成で表現することができることに注意してください。
実は、合成関数$g\circ f$の微分は次のように、部品となった関数$f, g$の微分で表現することができます。

$$
    \frac{d}{dx}(g\circ f)(x) = \frac{dg}{dx}f(x) + g(x)\frac{df}{dx}
$$

ニューラルネットワークも上記のような簡単な関数の合成で構築されているので、解析的に微分を計算することが可能です。